## Notebook that takes dihadron ntuples and plots kinematics distributions also with MC

In [ ]:
import ROOT
from ROOT import TFile
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import copy
import pandas as pd 
import time

import seaborn as sns

import root_pandas as rpd
from root_pandas import read_root

#mpl.rcParams['figure.figsize'] = 6.2,3.5
#mpl.rcParams['axes.labelsize'] = 17
#mpl.rcParams['lines.linewidth'] = 2
#mpl.rcParams['lines.markersize'] = 6
#mpl.rcParams['legend.fontsize']= 13

## Get data

In [ ]:
#this needs to be read in chunks otherwise it uses all memory. (from https://github.com/scikit-hep/root_pandas)
def getdatainChunks(filename,treename):
    dataframe =pd.DataFrame()
    for df in read_root(filename, treename, chunksize=100000) :
        #print df.shape[0]
        dataframe = pd.concat([dataframe,df])
    
    print (dataframe.shape[0])
    return dataframe
    

In [ ]:
df = {}
df_trigger = {}

for target in ['C','Fe','Pb']:
    print ('Getting dataframes from target %s'%target)
    df[target]=getdatainChunks('Pairs_%s.root'%target, target)
    df['%s_mix'%target] = getdatainChunks('Pairs_%s.root'%target, '%s_mix'%target)
    df['D_%s'%target]= getdatainChunks('Pairs_%s.root'%target, 'D_%s'%target)
    df['D_%s_mix'%target] = getdatainChunks('Pairs_%s.root'%target, 'D_%s_mix'%target)
    
    print ('Get trigger dataframes from target%s'%target)
    df_trigger['%s_trigger'%target] = getdatainChunks('Pairs_%s.root'%target, '%s_trigger'%target)
    df_trigger['D_%s_trigger'%target] = getdatainChunks('Pairs_%s.root'%target, 'D_%s_trigger'%target)


## GiBUU

In [ ]:
for target in ['D','C','Fe','Pb']:#,'Fe','C']:
    #GiBUU
    print target
    print '----pairs'
    df['GiBUU_%s'%target]= getdatainChunks('GiBUU_Pairs_%s.root'%target, target)
    print '----trigger'
    df_trigger['GiBUU_%s_trigger'%target]= getdatainChunks('GiBUU_Pairs_%s.root'%target, '%s_trigger'%target)

## Combine deuterium datasets

In [ ]:
df['D'] = pd.concat([df['D_Pb'],df['D_Fe'],df['D_C']]) #simply combine the deuterium results for all targets
df['D_mix'] = pd.concat([df['D_Pb_mix'],df['D_Fe_mix'],df['D_C_mix']]) #simply combine the deuterium results for all targets
df_trigger['D_trigger'] = pd.concat([df_trigger['D_Pb_trigger'],df_trigger['D_Fe_trigger'],df_trigger['D_C_trigger']]) #simply combine the deuterium results for all targets



In [ ]:
def applyCut(inputDataframe, cut, text=None):
    nbeforecut = inputDataframe.shape[0]
    cutDataframe = inputDataframe.query(cut)
    if text:
        print text, cutDataframe.shape[0], ' (%2.2f '%(100.0*cutDataframe.shape[0]/nbeforecut), '%)'
    return cutDataframe
def applyCuts(fullDataframe,name='default',isMC=False): 
    dataframe = fullDataframe
    print 'Entries before cut ', dataframe.shape[0]
    dataframe.eval('inelasticity = Nu/5.014', inplace=True)
    dataframe.eval('E = z*Nu', inplace=True)
    dataframe.eval('P = sqrt(E*E-0.140*0.140)', inplace=True)
    
    dataframe = applyCut(dataframe, 'Q2>1.0', 'Q2>1.0 :')
    #dataframe = applyCut(dataframe, 'W>2.0', 'W>2.0 :')
    #dataframe = applyCut(dataframe, 'Nu>3.5', 'Nu>3.5 :')

    dataframe = applyCut(dataframe, 'z>0.5', 'z>0.5 :')
    dataframe = applyCut(dataframe, 'P <2.5 ', 'P<2.5 ')
    dataframe = applyCut(dataframe, 'inelasticity<0.85','inelasticity < 0.85')
    return dataframe

def applyCutsPair(fullDataframe,name='default'):
    print 'Starting election on dipion variables'
    dataframe = fullDataframe
    dataframe.eval('z_tot = z+z_asso', inplace=True)
    dataframe.eval('E_asso = z_asso*Nu', inplace=True)
    dataframe.eval('P_asso = sqrt(E_asso*E_asso-0.140*0.140)', inplace=True)

    dataframe = applyCut(dataframe, 'pid*pid_asso<0', 'Opposite sign pairs')
    dataframe = applyCut(dataframe, 'P_asso>0.200', 'P_asso > 200 MeV')
    dataframe = applyCut(dataframe, 'P_asso<2.5', 'P_asso <2.5 GeV')
    
    return dataframe

def printPairBreakdown(dataframe):
    
    print 'Pairs with Leading pi+', dataframe.query('pid==211').shape[0]
    print 'Pairs with Leading pi-', dataframe.query('pid==-211').shape[0]
    print 'Pairs with Sub-Leading pi+', dataframe.query('pid_asso==211').shape[0]
    print 'Pairs with Sub-Leading pi-', dataframe.query('pid_asso==-211').shape[0]
    print 'pi+ pi+ pairs',dataframe.query('pid==211 and pid_asso==211').shape[0]
    print 'pi- pi- pairs',dataframe.query('pid==-211 and pid_asso==-211').shape[0]
    print 'pi+ pi- pairs',dataframe.query('pid==211 and pid_asso==-211').shape[0]
    print 'pi- pi+ pairs',dataframe.query('pid==-211 and pid_asso==211').shape[0]
    print '//////////////////////////////////////////////////////'
    return 


## apply cuts

In [ ]:
for key in df.keys():
    print key
    df[key] = applyCuts(df[key])
    #printPairBreakdown(df[key])
    df[key] = applyCutsPair(df[key])
    print '--------------'
    
for key in df_trigger.keys():
    print key
    df_trigger[key] = applyCuts(df_trigger[key])

## Event Variables

In [ ]:
def plotEventVariables(df):
    fig = plt.figure( figsize=(20, 10))
    plt.subplot(2,3,1)
    plt.hist(df['Nu'], bins=25,range =(2,5), histtype='step', normed=True,label='Data')
    plt.legend()
    plt.xlabel("Nu [GeV]", fontsize=20)

    plt.subplot(2,3,2)
    plt.hist(df['Q2'], bins=25,range =(1,5), histtype='step', normed=True,label='Data')
    #plt.hist(MC_reco['Q2'], bins=25,range = (1,5),  normed = True,alpha=0.2,label='MC')
    plt.xlabel("$Q^{2} [GeV^{2}]$", fontsize=20)

    #plt.subplot(2,3,3)
    #plt.hist(df['W'], bins=25,range =(1,3), histtype='step', normed=True,label='Data')
    #plt.hist(MC_reco['W'], bins=25,range = (1,3), normed = True,alpha=0.2,label='MC')
    #plt.xlabel("$W$ [GeV]", fontsize=20)

    plt.subplot(2,3,4)
    plt.hist(df['Xb'], bins=25,range =(0,1), histtype='step', normed=True,label='Data')
    #plt.hist(MC_reco['Xb'], bins=25,range = (0,1),  normed = True,alpha=0.2,label='MC')
    plt.xlabel("$x$", fontsize=20)
    
    plt.subplot(2,3,5)
    plt.hist(df['inelasticity'], bins=25,range =(0,1), histtype='step', normed=True,label='Data')
    plt.legend()
    plt.xlabel("inelasticity", fontsize=20)
    plt.axvline(x=.850,color='red',linestyle='--')


    #plt.subplot(2,3,5)
    #plt.hist(df['SectorEl'], bins = 12, range = (-0.5,5.5),histtype='step', normed=True,label='Data')
    #plt.hist(MC_reco['SectorEl'], bins = 12, range=(-0.5,5.5), normed = True,alpha=0.2,label='MC')
    #plt.xlabel('Sector El')

    #plt.subplot(2,3,6)
    #plt.hist(df['deltaZ'], bins = 100, range=(-50,50), histtype='step', normed=True,label='Data')
    #plt.hist(MC_reco['deltaZ'], bins = 100, range=(-50,50), normed = True,alpha=0.2,label='MC')
    #plt.xlabel('Delta z')
    #plt.show()
    
    return

In [ ]:
plt.hexbin(df_trigger['D_trigger']['Q2'],df_trigger['D_trigger']['Nu'], cmap='inferno')
plt.hexbin(df_trigger['GiBUU_D_trigger']['Q2'],df_trigger['GiBUU_D_trigger']['Nu'],cmap='inferno')


In [ ]:
plt.hexbin(df_trigger['GiBUU_D_trigger']['Q2'],df_trigger['GiBUU_D_trigger']['Nu'],cmap='inferno')


#### This is accomplished with sns.regplot (seaborn package needed). It plots the average of a quantity in bins o the other quantities. This is more useful than a 2D distribution

## Profile distributions to see correlation with kinematic variables with z2

In [ ]:
fig = plt.figure( figsize=(14, 5))

for ivar,variable in enumerate(['Q2','Xb','Nu','W']):
    plt.subplot(1,4,ivar+1)
    query = '3.0 < Nu<3.5 and P_asso>0.250'
    for i, target in enumerate(['D','C','Fe','Pb']):
        sns.regplot(x=df[target].query(query)['z_asso'],y=df[target].query(query)[variable],x_bins=np.logspace(np.log10(0.06), np.log10(0.5),12),fit_reg=None,label=target)
    
    for i, target in enumerate(['GiBUU_D','GiBUU_C','GiBUU_Fe','GiBUU_Pb']):
        sns.regplot(x=df[target].query(query)['z_asso'],y=df[target].query(query)[variable],x_bins=np.logspace(np.log10(0.06), np.log10(0.5),12),fit_reg=None,label=target)
   
    
    plt.legend(loc='best',frameon=False)

plt.xlim([0.0,0.5])
plt.tight_layout()
plt.savefig('ProfilePlots_1.png')
plt.show()

In [ ]:
fig = plt.figure( figsize=(14, 5))

for ivar,variable in enumerate(['z','P','P_asso']):
    plt.subplot(1,4,ivar+1)                                       
    for i, target in enumerate(['D','C','Fe','Pb']):
        sns.regplot(x=df[target]['z_asso'],y=df[target][variable],x_bins=np.logspace(np.log10(0.05), np.log10(0.5),12),fit_reg=None,label=target)
    
    for i, target in enumerate(['GiBUU_D','GiBUU_C','GiBUU_Fe','GiBUU_Pb']):
        sns.regplot(x=df[target]['z_asso'],y=df[target][variable],x_bins=np.logspace(np.log10(0.05), np.log10(0.5),12),fit_reg=None,label=target)
   
    
    plt.legend(loc='best',frameon=False)

plt.xlim([0.0,0.5])
plt.tight_layout()
plt.savefig('ProfilePlots_2.png')
plt.show()

In [ ]:


                               
for ivar,variable in enumerate(['xf','xf_asso','y','y_asso','dy']):  
    plt.subplot(1,5,ivar+1)                   

    for i, target in enumerate(['D','C','Fe','Pb']):
        sns.regplot(x=df[target]['z_asso'],y=df[target][variable],x_bins=np.logspace(np.log10(0.05), np.log10(0.5),12),fit_reg=None,label=target)
    plt.legend(loc='best',frameon=False)

plt.xlim([0.0,0.5])
plt.tight_layout()
plt.savefig('ProfilePlots_3.png')     
plt.clf() 
                               
for ivar,variable in enumerate(['mass','missing_mass','dipion_pt','dphi']): 
    plt.subplot(1,4,ivar+1)                   
    for i, target in enumerate(['D','C','Fe','Pb']):
        sns.regplot(x=df[target]['z_asso'],y=df[target][variable],x_bins=np.logspace(np.log10(0.05), np.log10(0.5),12),fit_reg=None,label=target)
    plt.legend(loc='best',frameon=False)

plt.xlim([0.0,0.5])
plt.tight_layout()
plt.savefig('ProfilePlots_4.png')    
plt.clf() 
                               
           
for ivar,variable in enumerate(['phi_lab_asso','theta_lab_asso','phi_lab','theta_lab']): 
    plt.subplot(1,4,ivar+1)                   
    for i, target in enumerate(['D','C','Fe','Pb']):
        sns.regplot(x=df[target]['z_asso'],y=df[target][variable],x_bins=np.logspace(np.log10(0.05), np.log10(0.5),12),fit_reg=None,label=target)
    plt.legend(loc='best',frameon=False)

plt.xlim([0.0,0.5])
plt.tight_layout()
plt.savefig('ProfilePlots_5.png')    

## plot for mixed events. In this case, key correlation should go away


## Profile plots but only for z1+z2<0.8

In [ ]:
fig = plt.figure( figsize=(21, 21))

for ivar,variable in enumerate(['Q2','Nu','Xb','W','z','P','P_asso','mass','missing_mass','dy','dipion_pt','xf','xf_asso','y','y_asso','phi_pq']):
    plt.subplot(4,4,ivar+1)
    for i, target in enumerate(['Pb','D']):
        sns.regplot(x=df[target].query('z_tot<0.8')['z_asso'],y=df[target].query('z_tot<0.8')[variable],x_bins=np.logspace(np.log10(0.05), np.log10(0.5),9),fit_reg=None,label=target)
    plt.legend(loc='best',frameon=False)

plt.xlim([0.0,0.5])
plt.savefig('ProfilePlots_withztotcut.png')

## invariant mass distribution

In [ ]:
fig, axs = plt.subplots(3, 4,sharex=True, sharey=True,figsize=(14,9), gridspec_kw={'wspace':0})

for i, target in enumerate(['D','C','Fe','Pb']):
    axs[0,i].hist(df[target]['mass'], bins=50, histtype='step', range=(0,2.0),normed=True,label='%s'%target,linewidth=2.0)
    axs[0,i].hist(df['%s_mix'%target]['mass'], bins=50, range=(0,2.0),normed=True,alpha=0.4,label='%s mixed'%target)
    axs[0,i].axvline(x=0.770,linestyle='--',color='black',alpha=0.6)
    axs[0,i].xaxis.set_label_text("Dipion Mass  [GeV]", fontsize=13)
    axs[0,i].legend(loc='best',frameon=False)

axs[0,0].yaxis.set_label_text("entries (normalized)", fontsize=13)


for i, target in enumerate(['D','C','Fe','Pb']):
    axs[1,i].hist(df[target].query('z_tot<0.8')['mass'], bins=50, histtype='step', range=(0,2.0),normed=True,label='%s'%target,linewidth=2.0)
    axs[1,i].hist(df['%s_mix'%target].query('z_tot<0.8')['mass'], bins=50, range=(0,2.0),normed=True,alpha=0.4,label='%s mixed'%target)
    axs[1,i].axvline(x=0.770,linestyle='--',color='black',alpha=0.6)
    axs[1,i].xaxis.set_label_text("Dipion Mass  [GeV]", fontsize=13)
    axs[1,i].legend(loc='best',frameon=False)

for i, target in enumerate(['D','C','Fe','Pb']):
    axs[2,i].hist(df[target].query('z_tot>0.8')['mass'], bins=50, histtype='step', range=(0,2.0),normed=True,label='%s'%target,linewidth=2.0)
    axs[2,i].hist(df['%s_mix'%target].query('z_tot>0.8')['mass'], bins=50, range=(0,2.0),normed=True,alpha=0.4,label='%s mixed'%target)
    axs[2,i].axvline(x=0.770,linestyle='--',color='black',alpha=0.6)
    axs[2,i].xaxis.set_label_text("Dipion Mass  [GeV]", fontsize=13)
    axs[2,i].legend(loc='best',frameon=False)

plt.tight_layout()
plt.savefig('Massforztotbins.png')    
                               

## Missing mas distribution

In [ ]:
fig, axs = plt.subplots(1, 4,sharex=True, sharey=True,figsize=(14,4), gridspec_kw={'wspace':0})

for i, target in enumerate(['D','C','Fe','Pb']):
    axs[i].hist(df[target]['missing_mass'], bins=50, histtype='step', range=(0,2.0),normed=True,label='all',linewidth=2.0)
    axs[i].hist(df[target].query('z_tot<0.8')['missing_mass'], bins=50, histtype='step', range=(0,2.0),normed=True,label='z1+z2<0.8',linewidth=2.0)
    axs[i].hist(df[target].query('z_tot>0.8')['missing_mass'], bins=50, histtype='step', range=(0,2.0),normed=True,label='z1+z2>0.8',linewidth=2.0)

    #axs[i].hist(df['%s_mix'%target]['missing_mass'], bins=50, range=(0,2.0),normed=True,alpha=0.4,label='%s mixed'%target)
    axs[i].axvline(x=0.938,linestyle='--',color='black',alpha=0.6)
    axs[i].xaxis.set_label_text("Missing mass  [GeV]", fontsize=13)
    axs[i].set_title(target)
    axs[i].legend(loc='best',frameon=False)

axs[0].yaxis.set_label_text("pdf", fontsize=13)


plt.tight_layout()
plt.savefig('MissingMassforztotbins.png')    

## Z and z_asso distributions without z1+z2 selection

In [ ]:
fig, axs = plt.subplots(1, 2,sharey=True,figsize=(14,5), gridspec_kw={'wspace':0})

axs[0].hist(df['D']['z_asso'], bins=30,range =(0,0.5), normed=True,label='D',alpha=0.5)

for i, target in enumerate(['C','Fe','Pb']):
    axs[0].hist(df[target]['z_asso'], bins=30,range =(0,0.5), histtype='step', normed=True,label=[target])
    
axs[1].hist(df['D']['z'], bins=50,range =(0.5,1.0), normed=True,label='D',alpha=0.5)
for i, target in enumerate(['C','Fe','Pb']):
    axs[1].hist(df[target]['z'], bins=50,range =(0.5,1.0), histtype='step', normed=True,label=[target]) 
 

axs[0].xaxis.set_label_text("$z_{2}$", fontsize=13)
axs[1].xaxis.set_label_text("$z_{1}$", fontsize=13)
axs[0].yaxis.set_label_text("entries (normalized)", fontsize=13)

plt.legend(loc='best',frameon=False)

plt.savefig('Zdistributions_all.png')    

## Z and z_asso distributions for z1+z2<0.8

In [ ]:
fig, axs = plt.subplots(1, 2,sharey=True,figsize=(14,5), gridspec_kw={'wspace':0})

axs[0].hist(df['D'].query('z_tot<0.8')['z_asso'], bins=30,range =(0,0.5), normed=True,label='D',alpha=0.5)

for i, target in enumerate(['C','Fe','Pb']):
    axs[0].hist(df[target].query('z_tot<0.8')['z_asso'], bins=30,range =(0,0.5), histtype='step', normed=True,label=[target])
    
axs[1].hist(df['D'].query('z_tot<0.8')['z'], bins=50,range =(0.5,1.0), normed=True,label='D',alpha=0.5)
for i, target in enumerate(['C','Fe','Pb']):
    axs[1].hist(df[target].query('z_tot<0.8')['z'], bins=50,range =(0.5,1.0), histtype='step', normed=True,label=[target]) 
 

axs[0].xaxis.set_label_text("$z_{2}$", fontsize=13)
axs[1].xaxis.set_label_text("$z_{1}$", fontsize=13)
axs[0].yaxis.set_label_text("entries (normalized)", fontsize=13)

plt.legend(loc='best',frameon=False)
plt.savefig('Zdistributions_cut.png')    

## Angular distributions phi_lab eta_lab for all z2 ranges

In [ ]:
fig = plt.figure( figsize=(12, 4))

plt.subplot(1,2,1)
plt.hexbin(df['Pb']['phi_lab'],df['Pb']['theta_lab'],cmap='inferno')
plt.xlabel("$\phi_{lab}$ (rad)", fontsize=14)
plt.ylabel("$\Theta_{lab}$ (rad) ", fontsize=14)
plt.title('Leading pion, $z_{1}$> 0.5')

plt.subplot(1,2,2)
plt.hexbin(df['Pb']['phi_lab_asso'],df['Pb']['theta_lab_asso'],cmap='viridis')
plt.xlabel("$\phi_{lab}$ (rad)", fontsize=14)
plt.ylabel("$\Theta_{lab}$ (rad) ", fontsize=14)
plt.title('Subleading pion, $z_{1}$> 0.5')

plt.savefig('Angles_leadingandsubleading.png')    



## angles for z1 for various z2 bins

In [ ]:
fig = plt.figure( figsize=(12, 12))

fig, axs = plt.subplots(1, 4,figsize=(14,4),sharey=True,sharex=True, gridspec_kw={'wspace':0, 'hspace':0})


asso_Edges = [0.05,0.08, 0.15,0.3,0.5]
asso_bins = [(x, y) for (x, y) in zip(asso_Edges[:-1], asso_Edges[1:])]

for j, asso_bin in enumerate(asso_bins):
    query_trigger = 'z> %2.2f and z<= %2.2f and theta_lab<1.0'%(0.5,1.0)
    query_asso = 'z_asso>%2.2f and z_asso<=%2.2f'%(asso_bin[0],asso_bin[1])
    query = query_trigger + ' and ' + query_asso
    #plt.subplot(3,3,j+1)
    axs[j].hexbin(df['D'].query(query)['phi_lab'],df['D'].query(query)['theta_lab'],cmap='inferno')
    axs[j].xaxis.set_label_text("$\phi_{lab}$ (rad)", fontsize=14)
    axs[0].yaxis.set_label_text("$\Theta_{lab}$  (rad)", fontsize=14)
    axs[j].set_title('$z_{1}$> 0.5 \n %2.2f < $z_{2}$ < %2.2f'%(asso_bin[0],asso_bin[1]), fontsize=13)
    

plt.savefig('z1LabAngles_z2bins_allpions.png')    
plt.show()

## Angle for z2 in different bins

In [ ]:
fig = plt.figure( figsize=(12, 12))

fig, axs = plt.subplots(1, 4,figsize=(14,4),sharey=True,sharex=True, gridspec_kw={'wspace':0, 'hspace':0})


asso_Edges = [0.05,0.08, 0.15,0.3,0.5]
asso_bins = [(x, y) for (x, y) in zip(asso_Edges[:-1], asso_Edges[1:])]

for j, asso_bin in enumerate(asso_bins):
    query_trigger = 'z> %2.2f and z<= %2.2f '%(0.5,1.0)
    query_asso = 'z_asso>%2.2f and z_asso<=%2.2f'%(asso_bin[0],asso_bin[1])
    #plt.subplot(3,3,j+1)
    axs[j].hexbin(df['D'].query(query_asso)['phi_lab_asso'],df['D'].query(query_asso)['theta_lab_asso'],cmap='viridis')
    axs[j].xaxis.set_label_text("$\phi_{lab}$ (rad)", fontsize=14)
    axs[0].yaxis.set_label_text("$\Theta_{lab}$  (rad)", fontsize=14)
    axs[j].set_title('$z_{1}$> 0.5 \n %2.2f < $z_{2}$ < %2.2f'%(asso_bin[0],asso_bin[1]), fontsize=13)
    

plt.savefig('LabAngles_z2bins_allpions.png')    
plt.show()

## Same as above but separately for pi+ and pi- of associated hadron

In [ ]:
fig = plt.figure( figsize=(12, 12))

fig, axs = plt.subplots(2, 4,figsize=(14,8),sharey=True,sharex=True, gridspec_kw={'wspace':0, 'hspace':0})


asso_Edges = [0.05,0.08, 0.15,0.3,0.5]
asso_bins = [(x, y) for (x, y) in zip(asso_Edges[:-1], asso_Edges[1:])]

for j, asso_bin in enumerate(asso_bins):
    query_trigger = 'z> %2.2f and z<= %2.2f '%(0.5,1.0)
    query_asso = 'z_asso>%2.2f and z_asso<=%2.2f'%(asso_bin[0],asso_bin[1])
    axs[0,j].hexbin(df['D'].query(query_asso + ' and pid_asso>0')['phi_lab_asso'],df['D'].query(query_asso + ' and pid_asso>0')['theta_lab_asso'],cmap='viridis')
    axs[1,j].hexbin(df['D'].query(query_asso + ' and pid_asso<0')['phi_lab_asso'],df['D'].query(query_asso + ' and pid_asso<0')['theta_lab_asso'],cmap='viridis')
    axs[1,j].xaxis.set_label_text("$\phi_{lab}$ (rad)", fontsize=14)
    axs[0,0].yaxis.set_label_text("$\Theta_{lab}$  Positive Pions (rad)", fontsize=14)
    axs[1,0].yaxis.set_label_text("$\Theta_{lab}$  Negative Pions (rad)", fontsize=14)
    axs[0,j].set_title('$z_{1}$> 0.5 \n %2.2f < $z_{2}$ < %2.2f'%(asso_bin[0],asso_bin[1]), fontsize=13)

plt.savefig('LabAngles_z2bins_piplusminus.png')    

plt.show()

## dphi_lab vs phi_lab for same-event and mixed-events

In [ ]:
def plotacceptance(df,df_mix,title='',lab=True):
    if(lab==True):
        var1 = 'phi_lab'
        var1_asso = 'phi_lab_asso'
        var2 = 'dphi_lab'
        label = 'lab'
    else:
        var1 = 'phi_pq'
        var1_asso = 'phi_pq_asso'
        var2 = 'dphi'
        label ='PQ'
    fig = plt.figure( figsize=(12, 9))
    plt.subplot(2,3,1)
    plt.title(title)
    plt.hexbin(df[var1],df[var2],cmap='inferno',gridsize=25)
    plt.xlabel("$\phi_{%s}$ (rad)"%label, fontsize=14)
    plt.ylabel("$|d\phi_{%s}|$ (rad) "%label, fontsize=14)

    plt.subplot(2,3,2)
    plt.hist(df[var1],bins=100,label='z1',alpha=0.5)
    plt.hist(df[var1_asso],bins=100,label='z2',alpha=0.5)
    plt.legend(loc='best',frameon=False)

    plt.xlabel("$\phi_{%s}$ (rad)"%label, fontsize=14)
    plt.title(title)

    plt.subplot(2,3,3)
    plt.hist(df[var2],bins=50)
    plt.xlabel("$|d\phi_{%s}|$ (rad)"%label, fontsize=14)
    plt.title(title)
    #mixed events
    plt.subplot(2,3,4)
    plt.hexbin(df_mix[var1],df_mix[var2],cmap='inferno',gridsize=25)
    plt.xlabel("$\phi_{%s}$ (rad)"%label, fontsize=14)
    plt.ylabel("$|d\phi_{%s}|$ (rad)"%label, fontsize=14) 
    #plt.title(title)

    plt.subplot(2,3,5)
    plt.hist(df_mix[var1],bins=100,label='z1',alpha=0.5)
    plt.hist(df_mix[var1_asso],bins=100,label='z2',alpha=0.5)
    plt.legend(loc='best',frameon=False)
    plt.xlabel("$\phi_{%s}$ (rad)"%label, fontsize=14)
    #plt.title(title)

    plt.subplot(2,3,6)
    plt.hist(df_mix[var2],bins=50)
    plt.xlabel("$|d\phi_{%s}|$ (rad)"%label, fontsize=14)
    #plt.title(title)

    plt.show()


## Example for 0.1<z2 < 0.2 for both lab and pq frames

In [ ]:
plotacceptance(df['D'].query('z_asso>0.1 and z_asso<0.2'),df['D_mix'].query('z_asso>0.1 and z_asso<0.2'),title='$0.1<z_2 <0.2$')
plotacceptance(df['D'].query('z_asso>0.1 and z_asso<0.2'),df['D_mix'].query('z_asso>0.1 and z_asso<0.2'),title='$0.1<z_2 <0.2$',lab=False)

## dphi_pq vs phi_pq for same-event and mixed-events

In [ ]:
asso_Edges = [0.05,0.08, 0.15,0.3,0.5]
asso_bins = [(x, y) for (x, y) in zip(asso_Edges[:-1], asso_Edges[1:])]

for j, asso_bin in enumerate(asso_bins):
    query_asso = 'z_asso>%2.2f and z_asso<=%2.2f'%(asso_bin[0],asso_bin[1]) 
    label =  '$%2.2f<z_2<%2.2f$'%(asso_bin[0],asso_bin[1]) 
    plotacceptance(df['D'].query(query_asso),df['D_mix'].query(query_asso),title=label,lab=False)

## z1 vs angles in lab / z2 vs angles in lab


In [ ]:
fig = plt.figure( figsize=(12, 8))
plt.subplot(2,2,1)
plt.hexbin(df['D']['z'],df['D']['theta_lab'], bins='log',cmap='viridis')
plt.xlabel("$z_{1}$", fontsize=14)
plt.ylabel("$\Theta_{lab}$ leading (rad) ", fontsize=14)

plt.subplot(2,2,2)
plt.hexbin(df['D']['z'],df['D']['phi_lab'], bins='log',cmap='viridis')
plt.xlabel("$z_{1}$", fontsize=14)
plt.ylabel("$\phi_{lab}$ (rad) ", fontsize=14)

plt.subplot(2,2,3)
plt.hexbin(df['D']['z_asso'],df['D']['theta_lab_asso'], bins='log',cmap='inferno')
plt.xlabel("$z_2$", fontsize=14)
plt.ylabel("$\Theta_{lab}$ subleading (rad) ", fontsize=14)

plt.subplot(2,2,4)
plt.hexbin(df['D']['z_asso'],df['D']['phi_lab_asso'], bins='log',cmap='inferno')
plt.xlabel("$z_2$", fontsize=14)
plt.ylabel("$\phi_{lab}$ (rad) ", fontsize=14)


plt.savefig('CorrelationAnglesandz1z2.png')    


## Same as above but sorted for pions of positive or negative charge

In [ ]:
fig = plt.figure( figsize=(12, 8))
plt.subplot(2,2,1)
plt.hexbin(df['D'].query('pid_asso>0')['z_asso'],df['D'].query('pid_asso>0')['theta_lab_asso'], bins='log',cmap='inferno')
plt.xlabel("$z_2$", fontsize=14)
plt.ylabel("$\Theta_{lab}$ subleading (rad) ", fontsize=14)

plt.subplot(2,2,2)
plt.hexbin(df['D'].query('pid_asso>0')['z_asso'],df['D'].query('pid_asso>0')['phi_lab_asso'], bins='log',cmap='inferno')
plt.xlabel("$z_2$", fontsize=14)
plt.ylabel("$\phi_{lab}$ (rad) ", fontsize=14)

plt.subplot(2,2,3)
plt.hexbin(df['D'].query('pid_asso<0')['z_asso'],df['D'].query('pid_asso<0')['theta_lab_asso'], bins='log',cmap='inferno')
plt.xlabel("$z_2$", fontsize=14)
plt.ylabel("$\Theta_{lab}$ subleading (rad) ", fontsize=14)

plt.subplot(2,2,4)
plt.hexbin(df['D'].query('pid_asso<0')['z_asso'],df['D'].query('pid_asso<0')['phi_lab_asso'], bins='log',cmap='inferno')
plt.xlabel("$z_2$", fontsize=14)
plt.ylabel("$\phi_{lab}$ (rad) ", fontsize=14)


plt.savefig('CorrelationAnglesandz1z2.png')    
